In [1]:
import sys
sys.path.append('/source/main')

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
import time
from itertools import chain

import torch
from torch import nn
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from preprocess import preprocessor
from model_def.baseline import Baseline
from model_def.simple_but_huge import SimpleButHuge
from utils import pytorch_utils, text_utils
from data_for_train import my_dataset
from train import trainer

In [3]:
my_dataset.bootstrap()

INFO:root:Src vocab contains 3519 tokens
INFO:root:Tgt vocab contains 8758 tokens


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# del model
model = SimpleButHuge(src_word_vocab_size=len(my_dataset.voc_src.index2word),
                     tgt_word_vocab_size=len(my_dataset.voc_tgt.index2word))
model.to(device)
PRE_TRAINED_MODEL='/source/main/train/output/saved_models/SimpleButHuge/2019-05-04T01:40:37/470000.pt'
checkpoint = torch.load(PRE_TRAINED_MODEL, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
logging.info('Load pre-trained model from %s successfully', PRE_TRAINED_MODEL)

model.eval()
logging.info('Model architecture: \n%s', model)
logging.info('Total trainable parameters: %s', pytorch_utils.count_parameters(model))

INFO:root:Load pre-trained model from /source/main/train/output/saved_models/SimpleButHuge/2019-05-04T01:40:37/470000.pt successfully
INFO:root:Model architecture: 
SimpleButHuge(
  (input_embedding): Embedding(3519, 1024)
  (conv1): Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(1024, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3_bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.3)
  (conv4): Conv1d(2048, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4_bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv1d(1024, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv5_bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv6_bn): B

In [5]:
def docs2input_tensors(preprocessed_docs, device):
    seq_len = [len(doc.split()) for doc in preprocessed_docs]
    word_input = [my_dataset.voc_src.docs2idx([doc], equal_length=MAX_LEN)[0] for doc in preprocessed_docs]
    
    inputs = (word_input, seq_len)
    inputs = [np.array(i) for i in inputs]
    input_tensors = [torch.from_numpy(i) for i in inputs]
    input_tensors = [i.to(device) for i in input_tensors]
    return input_tensors

def replace_unk_tok(pred, src):
    pred = [p if p!='¶' else s for p, s in zip(pred.split(), src.split())]
    return ' '.join(pred)

def predict_batch(docs):
    input_tensors = docs2input_tensors(docs, device)
    predict_tensor = model.cvt_output(model(*input_tensors))
    predict_numpy = predict_tensor.cpu().numpy()
    
    translated_docs = my_dataset.voc_tgt.idx2docs(predict_numpy)
    translated_docs = [' '.join(pred_doc.split()[:len(src_doc.split())]) 
                       for src_doc, pred_doc in zip(docs, translated_docs)]
    translated_docs = [replace_unk_tok(pred, src) for pred, src in zip(translated_docs, docs)]
    return translated_docs

def predict_docs(docs, batch_size):
    return list(chain(*[predict_batch(docs[i: i+batch_size]) for i in range(0, len(docs), batch_size)]))


In [6]:
def is_alpha(tok):
    return all([c.isalpha() for c in tok])

In [7]:
def validing_prediction(docs, pred_docs):
    docs = [doc.split() for doc in docs]
    pred_docs = [doc.split() for doc in pred_docs]
    
    for doc, pred in zip(docs, pred_docs):
        for idx, (t1, t2) in enumerate(zip(doc, pred)):
            if is_alpha(t1) and (not is_alpha(t2)):
                pred[idx] = t1
    pred_docs = [' '.join(doc) for doc in pred_docs]
    return pred_docs

In [8]:
with open('/dataset/vietnamese_tone_prediction/test.txt', 'rt', encoding='utf-8') as i_f:
    lines = i_f.readlines()
lines = [l[:-1] for l in lines]
lines = [(l[:3], l[4:]) for l in lines]
df_submit = pd.DataFrame(lines)

In [9]:
df_submit[df_submit[0]=='oPW']

,0,1
426,oPW,Anh: Quang Toan/BNEWS/TTXVN Thu truong Bo Giao thong Van tai Nguyen Van Cong vua ky cong van hoa toc de nghi UBND tinh Thai Nguyen som co van ban chinh thuc gui Bo Giao thong Van tai ve phuong an giam gia dich vu su dung duong bo da thong nhat voi nha dau tu du an BOT Thai Nguyen Cho Moi.


In [10]:
# HOT FIX
del my_dataset.voc_src.word2index['mollis']

In [11]:

pred_docs = preprocessor.infer_preprocess(df_submit[1], 200)
# df_submit[1] = validing_prediction(df_submit[1], pred_docs)
df_submit[1] = pred_docs

In [12]:
MAX_LEN = 200

In [13]:
predict_docs(['truyen co tich  tam cam'], batch_size=128)

['truyện cổ tích tấm cám']

In [14]:
MAX_LEN = 200
df_submit['pred'] = predict_docs(list(df_submit[1]), batch_size=32)

In [52]:
df_submit[df_submit[0]=='Mye']

,0,1,pred
610,Mye,"dieu nay khien nhieu nguoi ham mo khong khoi thac mac boitruoc do , `` nu hoang noi y '' va vu khac tiep tung la `` cap bai trung '' khi cung nhau xuat hien trong cac su kien lon , nho cung nhu cungxuat hien ben nhau trong cuoc song doi thuong .","điều này khiến nhiều người hâm mộ không khỏi thắc mắc boitruoc đó , `` nữ hoàng nội y '' và vũ khắc tiệp từng là `` cặp bài trùng '' khi cùng nhau xuất hiện trong các sự kiện lớn , nhỏ cũng như cungxuat hiện bên nhau trong cuộc sống đời thường ."


In [16]:
hot_patch_fix = df_submit['pred'].map(lambda x: x.find('vốn cuốn biểu ,') != -1)

In [17]:
# df_submit.iloc[hot_patch_fix, 2] = 
df_submit.loc[hot_patch_fix, 'pred'] = df_submit.loc[hot_patch_fix, 'pred'].map(lambda x: 'a a a a ' + x[16:])
df_submit.loc[7504, 'pred'] = 'a ' + df_submit.loc[7504, 'pred']

In [18]:
df_submit.loc[7504, 'pred']

'a a a a a dũng đã từng tổ chức nhiều trận đấu để ủng hộ đồng bào miền trung bị lũ lụt , người thân của nghệ sĩ bị bệnh hiểm nghèo . . . các trận đấu được đồng hành bởi nhiều doanh nghiệp yêu thích bóng đá và quan tâm đến các hoạt động cộng đồng .'

In [19]:
lines = df_submit[[0, 'pred']].to_dict('records')

In [20]:
lines = [item[0]+ ',' + item['pred'] + '\n' for item in lines]

In [21]:
lines[0]

'axS,trung quốc đã mở rộng ảnh hưởng của họ trong khu vực thông qua các bước leo thang ép buộc các nước láng giềng ở hoa đông , biển đông , bởi đắp đảo nhân tạo và quân sự hóa các cấu trúc địa lý trên biển đông trái luật pháp quốc tế ; tìm cách chia rẽ hoa kỳ khối các đồng minh châu á thông qua các nỗ lực ép buộc và lèo lái kinh tế .\n'

In [22]:
with open('my_prediction.txt', 'wt', encoding='utf-8') as o_f:
    o_f.writelines(lines)

In [23]:
text_utils.decompose_predicted_test_file('my_prediction.txt',
                                        'out_no_tone.txt', 'out_simplified.txt')

In [24]:
print('')

In [41]:
df_pred = pd.read_csv('out_no_tone.txt')
df_source = pd.read_csv('/dataset/vietnamese_tone_prediction/test_word_per_line.txt', dtype={'no_tone': str})

In [42]:
df_pred['no_tone'] = df_pred['no_tone'].map(str)
df_source['no_tone'] = df_source['no_tone'].map(str)

In [45]:
df_ = pd.merge(df_pred, df_source, how='outer', on='id', suffixes=('_pred', '_source'))

In [51]:
df_[df_['id'].map(lambda x: x[:3] == 'Mye')]

,id,no_tone_pred,no_tone_source
33627,Mye000,dieu,Dieu
33628,Mye001,nay,nay
33629,Mye002,khien,khien
33630,Mye003,nhieu,nhieu
33631,Mye004,nguoi,nguoi
33632,Mye005,ham,ham
33633,Mye006,mo,mo
33634,Mye007,khong,khong
33635,Mye008,khoi,khoi
33636,Mye009,thac,thac


In [48]:
df_[df_['no_tone_source'].isnull()]

,id,no_tone_pred,no_tone_source
26840,iWd015,month,NaN
33673,Mye046,doi,NaN
33674,Mye047,thuong,NaN
46359,ZLQ051,loa,NaN
53924,yIk039,hnm,NaN
65628,gbS080,rai,NaN
67163,kzN061,qua,NaN
91244,HbH075,chung,NaN
115147,GHB075,sinh,NaN
153269,bqf067,nuoc,NaN


In [34]:
x['no_tone_y'].isnull().sum()

0

In [40]:
x[x['no_tone_x'].isnull()]

,id,no_tone_x,no_tone_y
453446,iWd015,NaN,month
453447,Mye046,NaN,doi
453448,Mye047,NaN,thuong
453449,ZLQ051,NaN,loa
453450,yIk039,NaN,hnm
453451,gbS080,NaN,rai
453452,kzN061,NaN,qua
453453,HbH075,NaN,chung
453454,GHB075,NaN,sinh
453455,bqf067,NaN,nuoc


In [36]:
x1.shape

(453467, 2)

In [37]:
x2.shape

(453446, 2)

In [39]:
x1['no_tone'] == x2['no_tone']

ValueError: Can only compare identically-labeled Series objects

In [ ]:
x1['no_tone'].head()

In [ ]:
x2['no_tone'].head()